## Imports

In [31]:
import tensorflow as tf
from keras.layers.normalization import layer_normalization

In [32]:
from tensorflow import keras
from keras.layers.normalization import layer_normalization
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
import warnings
import numpy as np
# import matplotlib.pyplot as plt
import cv2
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint, EarlyStopping
import sklearn as sk
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
warnings.simplefilter(action='ignore', category=FutureWarning)

## Train and Test image directory paths

In [33]:
train_path = r'D:\gestures\train'
test_path = r'D:\gestures\test'

In [34]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=train_path, target_size=(64,64), class_mode='categorical', batch_size=10, shuffle=True)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=test_path, target_size=(64,64), class_mode='categorical', batch_size=10, shuffle=True)

Found 755 images belonging to 5 classes.
Found 755 images belonging to 5 classes.


## Building the model

In [35]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(64,64,3)))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'valid'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Flatten())

model.add(Dense(64,activation ="relu"))
model.add(Dense(128,activation ="relu"))
model.add(Dense(128,activation ="relu"))
model.add(Dense(5,activation ="softmax"))

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0001)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

model.compile(optimizer=SGD(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0005)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

history2 = model.fit(train_batches, epochs=100, callbacks=[reduce_lr, early_stop],  validation_data = test_batches)

Epoch 1/100
76/76 [==============================] - 5s 66ms/step - loss: 2.5785 - accuracy: 0.9086 - val_loss: 0.1983 - val_accuracy: 0.9404
Epoch 2/100
76/76 [==============================] - 3s 42ms/step - loss: 0.0064 - accuracy: 1.0000 - val_loss: 0.1999 - val_accuracy: 0.9430
Epoch 3/100
76/76 [==============================] - 3s 43ms/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.2002 - val_accuracy: 0.9417


In [36]:
history2.history

{'loss': [2.578505039215088, 0.006435680203139782, 0.003241561120375991],
 'accuracy': [0.9086092710494995, 1.0, 1.0],
 'val_loss': [0.19825880229473114, 0.1998724788427353, 0.20020121335983276],
 'val_accuracy': [0.940397322177887, 0.9430463314056396, 0.9417218565940857],
 'lr': [0.001, 0.001, 0.0005]}

## Saving our model

In [37]:
model.save('signModelNew')

INFO:tensorflow:Assets written to: signModelNew\assets


## Metrics

In [38]:
imgs, labels = next(test_batches) 
scores = model.evaluate(imgs, labels, verbose=0)
print('{0} of {1}; {2} of {3}%'
.format(model.metrics_names[0], scores[0], model.metrics_names[1], scores[1]*100))
scores

loss of 0.008135863579809666; accuracy of 100.0%


[0.008135863579809666, 1.0]

In [39]:
imgs, labels = next(test_batches)
model = keras.models.load_model("signModelNew")
# new_model = tf.keras.models.load_model('saved_model/my_model')
scores = model.evaluate(imgs, labels, verbose=0)
print('{0} of {1}; {2} of {3}%'
.format(model.metrics_names[0], scores[0], model.metrics_names[1], scores[1]*100))

loss of 0.024884339421987534; accuracy of 100.0%


In [40]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 31, 31, 64)        18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 13, 13, 128)       73856     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 6, 6, 128)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 4608)             

## Predictions

In [41]:
imgs, labels = next(test_batches) 

word_dict = {0:'One', 1:'Two', 2:'Three', 3:'I Love You', 4:'Little'}

predictions = model.predict(imgs, verbose=0)
predicted_labels = []
actual_labels = []

print("Predicted labels")
for ind, i in enumerate(predictions):
    print(word_dict[np.argmax(i)])
    predicted_labels.append(word_dict[np.argmax(i)])
    
print('Actual labels')
for i in labels:
    print(word_dict[np.argmax(i)])
    actual_labels.append(word_dict[np.argmax(i)])


Predicted labels
One
Two
Two
Three
Three
Three
Little
One
Two
One
Actual labels
One
Two
Two
Three
Three
Three
Little
One
Two
One


In [23]:
print ("Precision", sk.metrics.precision_score(actual_labels, predicted_labels, average='macro'))
print ("Recall", sk.metrics.recall_score(actual_labels, predicted_labels, average='macro'))
print ("f1_score", sk.metrics.f1_score(actual_labels, predicted_labels, average='macro'))
print ("confusion_matrix: ")
print (sk.metrics.confusion_matrix(actual_labels, predicted_labels))

Precision 0.9583333333333334
Recall 0.8333333333333334
f1_score 0.8666666666666666
confusion_matrix: 
[[1 0 0]
 [0 1 1]
 [0 0 7]]


In [ ]:
labels

In [ ]:
predictions